This notebook will select isotherms in the required temperature range and with the desired adsorbates from the sqlite database where all the ISODB isotherms are stored. To run this notebook the root directory must be the main folder.

In [2]:
import warnings
import numpy as np
import pandas as pd
import pygaps

%cd ..

Get all isotherms

In [3]:
# Database location
db_path = r"./data/iso.db"

# Filter warnings to remove pygaps unit warning
warnings.simplefilter('ignore')

# Get all isotherms
isotherms = pygaps.db_get_isotherms(db_path, {})

print('Initial isotherms', len(isotherms))

Selected 21261 isotherms
Initial isotherms 21261


Perform rough sorting by removing isotherms without enough datapoints, and with too high/low pressure and temperature.

In [4]:

isotherms = [i for i in isotherms if len(i.loading()) > 6]
print('Isotherms with enough points', len(isotherms))

isotherms = [i for i in isotherms if i.pressure()[-1] < 150]
print('After removing high pressure isotherms:', len(isotherms))

isotherms = [i for i in isotherms if i.pressure()[-1] > 0.05]
print('After removing low pressure isotherms:', len(isotherms))

isotherms = [i for i in isotherms if max(i.loading()) < 100]
print('After removing high loading isotherms:', len(isotherms))

isotherms = [i for i in isotherms if i.temperature < 423.15]
print('After removing high temperature isotherms:', len(isotherms))


Isotherms with enough points 20275
After removing high pressure isotherms: 19700
After removing low pressure isotherms: 18777
After removing high loading isotherms: 16771
After removing high temperature isotherms: 16652


Select isotherms with a subset of adsorbates and in a temperature range.

In [5]:
gases = [
    'hydrogen', 'neon', 'argon', 'krypton', 'xenon',
    'methane', 'ethane', 'ethene', 'acetylene',
    'propane', 'propene',
    'butane', 'isobutane',
    '1-butene', 'cis-2-butene', 'trans-2-butene',
    'isobutene',
    'isopentane',
    'carbon dioxide', 'sulphur dioxide', 'nitrogen dioxide',
    'oxygen', 'carbon monoxide', 'nitrogen',
    'benzene', 'toluene',
    'water', 'methanol', 'ethanol', 'ammonia',
]

tmin = 292
tmax = 313

sel_isotherms = [i for i in isotherms if
                 i.adsorbate in gases and
                 tmin < i.temperature < tmax]
print('Selected isotherms in T range:', len(sel_isotherms))


Selected isotherms in T range: 5887


Save all remaining isotherms as JSON files in the isotherm folder.

In [6]:
path = r"./data/isotherms/"
for iso in sel_isotherms:
    pygaps.isotherm_to_jsonf(iso, (path + iso.filename + '.json'))